In [1]:
import os

In [2]:
%pwd

'/mnt/e/bgRemoveApp/bg_removal/research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallBackConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path
    csv_filePath:Path    

In [5]:
from bgremove.constants import *
from bgremove.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        print(self.config.prepare_callbacks)
        
    def get_call_backs_config(self)->PrepareCallBackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallBackConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath),
            csv_filePath = Path(config.csv_path)
        )

        return prepare_callback_config

In [7]:
import os
import tensorflow as tf
import time

2023-08-29 18:20:09.910165: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 18:20:10.907589: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2023-08-29 18:20:12,148: INFO: utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2023-08-29 18:20:12,150: INFO: utils: NumExpr defaulting to 8 threads.]


In [8]:
params = read_yaml(PARAMS_FILE_PATH)

[2023-08-29 18:20:12,581: INFO: common: yaml file: params.yaml loaded successfully]


In [9]:
class PrepareCallBacks:
    def __init__(self, config:PrepareCallBackConfig):
        self.config= config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
            return tf.keras.callbacks.ModelCheckpoint(
                filepath=str(self.config.checkpoint_model_filepath),
                verbose=params.VERBOSE,
                save_best_only=params.SAVE_BEST_ONLY)
    
    @property
    def _reduce_on_paleatue(self):
            return tf.keras.callbacks.ReduceLROnPlateau(
                monitor=params.MONITOR,
                factor=params.FACTOR, 
                patience=params.PATIENCE_REDUCE_LEARNING, 
                min_lr=params.MIN_LR, 
                verbose=params.VERBOSE)
            
    @property
    def _csv_logger(self):
            return tf.keras.callbacks.CSVLogger(self.config.csv_filePath)
    
    @property 
    def _early_stopping(self):
            return tf.keras.callbacks.EarlyStopping(monitor=params.MONITOR, patience=params.PATIENCE_EARLY_STOPPING, restore_best_weights=params.RESTORE_BEST_WEIGHTS)
            
    def get_callbacks(self):
        return [
            self._create_tb_callbacks, 
            self._create_ckpt_callbacks,
            self._reduce_on_paleatue,
            self._csv_logger,
            self._early_stopping
        ]

In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_call_backs_config()
    prepare_callbacks = PrepareCallBacks(config= prepare_callbacks_config)
    callback_list = prepare_callbacks.get_callbacks()
    print(callback_list)
except Exception as e:
    raise e

[2023-08-29 18:20:12,623: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-08-29 18:20:12,635: INFO: common: yaml file: params.yaml loaded successfully]
{'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5', 'csv_path': 'artifacts/prepare_callbacks/csv/log.csv'}
[2023-08-29 18:20:12,648: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2023-08-29 18:20:12,663: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[<keras.callbacks.TensorBoard object at 0x7f85b6b10730>, <keras.callbacks.ModelCheckpoint object at 0x7f85b6b13dc0>, <keras.callbacks.ReduceLROnPlateau object at 0x7f85b6b13e80>, <keras.callbacks.CSVLogger object at 0x7f85b6b10460>, <keras.callbacks.EarlyStopping object at 0x7f85b6b10670>]


In [11]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-08-29-18-21-55'